# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loan CSV file generated from WeatherPy Folder
weather_data_to_load = "weather_df.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head(30)

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
1,vila do maio,CV,2020-04-14 13:04:48,15.13,-23.22,75.0,51.0,82.670,10.30
2,feijo,BR,2020-04-14 13:04:49,-8.16,-70.35,6.0,48.0,91.544,2.52
3,taywarah,AF,2020-04-14 13:04:49,33.35,64.42,54.0,40.0,39.956,1.38
4,upernavik,GL,2020-04-14 13:04:49,72.79,-56.15,100.0,94.0,24.404,3.98
5,ushuaia,AR,2020-04-14 13:00:01,-54.80,-68.30,100.0,76.0,48.470,2.58
6,butaritari,KI,2020-04-14 13:04:49,3.07,172.79,37.0,80.0,83.894,7.90
8,saldanha,ZA,2020-04-14 13:04:49,-33.01,17.94,0.0,68.0,64.670,3.60
9,punta arenas,CL,2020-04-14 13:00:00,-53.15,-70.92,75.0,62.0,53.870,8.70
10,hithadhoo,MV,2020-04-14 13:04:50,-0.60,73.08,49.0,64.0,85.964,0.82
11,bluff,NZ,2020-04-14 13:04:50,-46.60,168.33,73.0,87.0,43.268,1.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)


# Locations
locations = dropna_weather_data[["Latitude", "Longitude"]]

humidity = dropna_weather_data["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filter vacation with zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,nouadhibou,MR,2020-04-14 13:04:54,20.93,-17.03,0.0,53.0,71.870,9.80
1,giddalur,IN,2020-04-14 13:05:02,15.35,78.92,0.0,39.0,79.664,1.35
2,matay,EG,2020-04-14 13:05:25,28.42,30.78,0.0,21.0,71.870,4.38
3,umm lajj,SA,2020-04-14 13:05:26,25.02,37.27,0.0,49.0,71.348,2.78
4,tahta,EG,2020-04-14 13:05:54,26.77,31.50,0.0,16.0,77.270,5.70
5,mecca,SA,2020-04-14 13:04:24,21.43,39.83,0.0,38.0,70.664,3.13
6,puerto madryn,AR,2020-04-14 13:04:19,-42.77,-65.04,0.0,23.0,71.042,2.81


In [6]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel
0,nouadhibou,MR,2020-04-14 13:04:54,20.93,-17.03,0.0,53.0,71.870,9.80,El Medina
1,giddalur,IN,2020-04-14 13:05:02,15.35,78.92,0.0,39.0,79.664,1.35,Shivaji Lodge
2,matay,EG,2020-04-14 13:05:25,28.42,30.78,0.0,21.0,71.870,4.38,Nag Aespen
3,umm lajj,SA,2020-04-14 13:05:26,25.02,37.27,0.0,49.0,71.348,2.78,Tanal Hotel Apartment
4,tahta,EG,2020-04-14 13:05:54,26.77,31.50,0.0,16.0,77.270,5.70,Møhãmęd Tãręk 🏠
5,mecca,SA,2020-04-14 13:04:24,21.43,39.83,0.0,38.0,70.664,3.13,Al Shohada Hotel MAKKAH
6,puerto madryn,AR,2020-04-14 13:04:19,-42.77,-65.04,0.0,23.0,71.042,2.81,Hotel Tolosa


In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

Figure(layout=FigureLayout(height='420px'))